# Explaining the Predictions of a Convolutional Neural Network on Blood Smears with Different Methods

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Sulam-Group/h-shap/blob/jaco/chembe_ml/demo/BBBC041/explain_compare.ipynb)

## Task

To explain the predictions of a convolutional neural network trained to predict the presence of trophozoites (o.e., malaria infected cells) in human blood smears. The network was trained on the [BBBC041 dataset](https://data.broadinstitute.org/bbbc/BBBC041/).

## Requirements



## Load the model

In [ ]:
import torch

# Define the device to run on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained model
torch.set_grad_enabled(False)
model = torch.hub.load(
    "Sulam-Group/h-shap:jaco/chembe_ml", "bbbc041trophozoitenet", trust_repo="check"
)
model = model.to(device)
model.eval()